In [1]:
import cv2 
import numpy as np 
import pandas as pd 
import os
from sklearn import preprocessing 

In [118]:
raw =  {10:'left_ankel' , 13:'right_ankel' , 9:'left_knee',12:'right_knee',8:'left_hip',
        11:'right_hip', 4:'left_wrist',7:'right_wrist', 3:'left_elbow',6:'right_elbow',
        2:'left_shoulder',5:'right_shoulder',1:'nose',17:'right_ear',15:'right_eye',
        14:'left_eye' , 16:'left_ear', 0:'unknown'}
df_cols_to_keys = None 
sorted_keys = np.sort((np.array(list(raw.keys()))))
PART_IDS = {raw[i]:i for i in sorted_keys}

CONNECTED_PART_NAMES = [
    ("left_hip", "right_hip"), ("left_elbow", "left_shoulder"),
    ("left_elbow", "left_wrist"), ("left_hip", "left_knee"),
    ("left_knee", "left_ankel"), ("right_hip", "right_shoulder"),
    ("right_elbow", "right_shoulder"), ("right_elbow", "right_wrist"),
    ("right_hip", "right_knee"), ("right_knee", "right_ankel"),
    ("left_shoulder", "right_shoulder"), ("left_hip", "left_shoulder")
]
CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES];

UNCONNECTED_PART_NAMES = ["nose", "right_ear" , "left_ear", "right_eye" , "left_eye",  "unknown"]
UNCONNECTED_PART_INDICES = [PART_IDS[a] for a in UNCONNECTED_PART_NAMES];

ARGS = {'normDatasetPath':'/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPosesL2Normalized.csv',
        'datasetPath':'/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPoses.csv',
        'datasetGen':False}

#Handling Main Dataset

In [3]:
def load_data():
    with open('/Users/sandeep/Downloads/dataset/annotation_dict.json') as f:
      data = json.load(f)
    labels = {0 : "block", 1 : "pass", 2 : "run", 
              3: "dribble",4: "shoot",5 : "ball in hand", 
              6 : "defense", 7: "pick" , 8 : "no_action" ,
              9: "walk" ,10: "discard"}
    return data , labels

In [4]:
if ARGS['datasetGen']:
    annotations , labels = load_data()
def get_label(fileName):
    label = annotations[fileName]
    return labels[label]

In [5]:
 def getScaleFactors(imgW, imgH, originalImgH , originalImgW):
    '''
       Helper function for get_data()
       Returns a tensor with given scalefactor (wdith,height). 
    '''
    return (imgW/originalImgW, imgH/originalImgH);

In [6]:
def mapToSquareImage(coord,scaleFactors):
    '''
      Helper function for get_data()
      Returns coord (width, height)
    '''
    coordInOriginalImage = [int(coord[0] * scaleFactors[0]) , int(coord[1] * scaleFactors[1])]
    return tuple(coordInOriginalImage)

In [41]:
def get_extended_pose(poses_dict, label=None):
    '''
    Helper function for get_data() and get_posenet_extended_pose()
    
    Parameters
    ----------
    pose : A dictionary containing 1 pose. The keys refer to bodyparts relative to the raw dict.
    Example {0:(x,y), ....} 
    
    label : get_data() will send label arugment but get_posenet_extended_pose() will not
    Return
    ---------
    extended_pose : A dictionary containing x and y coord has it own key
    Example {0_x: 1 ,0_y: 7 ....}  
    '''
    extended_pose = {}
    scale = getScaleFactors(244,244,176,128)
    for key,value in poses_dict.items():
        #TO DO: scale the value before setting to the value
        value = mapToSquareImage(value,scale)
        extended_pose[f'{key}_x'] = value[0]
        extended_pose[f'{key}_y'] = value[1]
    if label != None:
        extended_pose['label'] = label 
    return extended_pose
        

In [8]:
def get_data():
    '''
    Helper function for generate_dataset()
    
    Returns
    -------
    data : A 1D list contining dict of 16 poses gotton from 16 frames for a single player
    Example [{0_x:1 , 0_y:2 , 1_x:67........,'label':'walk'] 
    The coordinates of the poses coords for players in sqaure image of size (244, 244)
    '''
    fileName = os.listdir('/Users/sandeep/Downloads/dataset/examples')
    fileName = [name for name in fileName if name[-1] != '4']
    data = []
    for name in fileName:
        posePath = f'/Users/sandeep/Downloads/dataset/examples/{name}'
        try:
            poses_list,label = np.load(posePath,allow_pickle=True) , get_label(name[0:-4])
            for poses_dict in poses_list:
                extended_pose = get_extended_pose(pose_dict,label=label)
                data.append(extended_pose)
        except KeyError:
            continue
    return data   

In [9]:
def get_keys(df_keys):
    '''
    Helper function for get_proxy_coord
    Returns keys that the df columns name refers too. eg. 0_x -> 0 
    '''
    global df_cols_to_keys 
    return [df_cols_to_keys[key] for key in df_keys]
    

In [10]:
def get_connected_keys(key):
    key_pair_tuple = [t for t in CONNECTED_PART_INDICES if key in t]
    return [key1 if key1 != key else key2 for (key1,key2) in key_pair_tuple ]

In [11]:
def filter_keys(list1 , list2):
    '''
    Helper function for get_proxy_coord
    Returns keys that are not in the list2
    '''
    return list(set(list1)-set(list2))

In [12]:
def map_keys_to_df_key(key):
    return f'{key}_x' , f'{key}_y'

In [13]:
def get_unconneted_proxy_coord(keys):
    '''
    Helper function for get_proxy_coord
    '''
    usable_unconnected_keys = filter_keys(UNCONNECTED_PART_INDICES , keys)
    if len(usable_unconnected_keys) != 0:
        key_x , key_y  = map_keys_to_df_key(usable_unconnected_keys[0])
        return key_x , key_y
    else:
        return get_connected_proxy_coord(2,keys)#trace starts from right shoulder

In [14]:
def get_connected_proxy_coord(key,keys):
    '''
    Helper function for get_proxy_coord
    '''
    connected_keys_with_key = get_connected_keys(key)
    usable_connected_keys = filter_keys(connected_keys_with_key,keys)
    if  len(usable_connected_keys) != 0:#base-case, we know there is no pose with 1 points
        key_x , key_y = map_keys_to_df_key(usable_connected_keys[0])
        return key_x,key_y
    else:
       key_x, key_y = get_connected_proxy_coord(connected_keys_with_key[0],keys)
       return key_x,  key_y

In [15]:
def get_proxy_coord(row):
    '''
    Helper function to replace the NaN values/ the poses that are not visible using poxy poses
    PARAMS key: the key that gets passed is eg.1_x
    '''
    global df_cols_to_keys 
    #df keys with nan
    df_keys = [key for key in row[row.isnull()].index.tolist() if key[-1] != 'y'] 
    keys = get_keys(df_keys)
    for key in keys:
        if key in UNCONNECTED_PART_INDICES:#if the nan key is a  Unconnected part 
            key_x , key_y = get_unconneted_proxy_coord(keys)
        else:#if the nan key is a  Connected part 
            key_x, key_y = get_connected_proxy_coord(key,keys)
                
        current_key_x , current_key_y = map_keys_to_df_key(key)
        row[current_key_x] , row[current_key_y] = row[key_x] , row[key_y]
    return row

In [16]:
def get_df_cols():
    '''
    Helper function for generate_dataset()
    '''
    cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    a = []
    for col in cols:
        xCol = f'{col}_x'
        yCol = f'{col}_y'   
        a.append(xCol)
        a.append(yCol)
    a.append('label')
    b = {}
    for col in cols:
        b[f'{col}_x'] = col
        b[f'{col}_y'] = col

    return a,b

In [17]:
def fillNaN(df):
    '''
    Helper function for generate_dataset()
    Updates the NaN cells with proxy coords. 
    '''
    return  df.apply(get_proxy_coord, axis=1)

In [122]:
def generate_dataset(data=None):
    '''
    1.This function will fill the missing bodypoints with proxy coords and
      generate l2 normalized dataset from raw data
    
    2.Helper function for generate_data_posenet()
    
    Parameters
    ----------
    data : A list with extended_poses.
    If data is not passed then you are aiming to generate the main pose dataset 
    Example : [{0_x:1, 0_y:1 ,17_y:NaN, 17_y:NaN} , {0_x:1, 0_y:1 ,16_y:NaN, 16_y:NaN} , {..}]
    
    Return
    -------
    normalized_data : A 2D array with normalized data 
    [[0_x, 0_y, 1_x, 1_y, .......17_y] , [......]]
    '''
    global df_cols_to_keys
    if ARGS['datasetGen']:
        data = get_data()
        df = pd.DataFrame(data)
    else:
      df = pd.DataFrame(data)
      df['0_x'] , df['0_y'] = np.nan , np.nan
        
    df_cols,df_cols_to_keysx = get_df_cols()
    df_cols_to_keys = df_cols_to_keysx
    df = df[df_cols[:-1]]
    df = fillNaN(df)
    normalized_data = preprocessing.normalize(df, axis=1)
    return normalized_data

In [19]:
def read_dataset(norm=True):
    if norm:
        return pd.read_csv(ARGS['normDatasetPath'])
    elif norm == False:
        return pd.read_csv(ARGS['datasetPath'])



#Handeling Posenet Data

In [95]:
def posenet_part_to_part(part):
    '''
    Helper function of get_posenet_extended_pose
    '''
    if 'Ankle' in part:
        part = part.replace('Ankle' , 'ankel')
    if 'right' in part: 
        return f'right_{part[5:].lower()}'
    elif 'left' in part:
        return f'left_{part[4:].lower()}'
    return part
           

In [53]:
def map_part_to_key(posenetPart):
    '''
    Helper function of get_posenet_extended_pose()
    '''
    for key, part in raw.items():
        if part == posenetPart:
            return key
    raise Exception(f'{posenetPart} not in raw')

In [22]:
def get_posenet_extended_pose(pose):
    '''
    Helper function for generate generate_dataset()
    
    Parameters
    ----------
    pose : A list containing dictionary containing 1 pose. The keys refer to bodyparts relative to the posenet pred.
    Example [{'score': 0.8984865546226501, 'part': 'nose', 'position': {'x': 741.3767104497117, 'y': 201.7590852932459}}..].
    
    Return
    ---------
    extended_pose : A dictionary containing x and y coord has it own key
    Example {0_x: 1 ,0_y: 7 ....}  
    '''
    #Change keys
    pose_dict = {}
    for part_dict in pose:
        part = posenet_part_to_part(part_dict['part']) 
        key = map_part_to_key(part)
        pose_dict[key] = (part_dict['position']['x'] , part_dict['position']['y'])
    extended_pose = get_extended_pose(pose_dict)
    return extended_pose


In [105]:
def add_unkown_keypoint(pose):
    '''
    NOT USED ANYMORE! THIS FUNCTION DOES NOT RESPECT THE DATA PIPELINE
    Helper function of generate_data_posenet()
    This function appends a new  bodypart "unknown" object to the human keypoints list.
    This is important because all the poses in posenet pred will not contain thi body part, however,
    our original dataset contained this part.
    
    Parameter
    ---------
    pose : A 1D list with a human body parts
    Example [{score: , part:.. ,position:..}, secondBodyPart..a..]
    
    Return
    ----------
    pospose : A 1D list with a human body parts with 'unknown' body object 
    Example [{score: , part:'uknow' ,position:..} , {score: , part:.. ,position:..}, secondBodyPart..a..]
    '''
    unknow_obj = {'score': 0.1,
                 'part': 'unknown',
                 'position': {'x': np.nan, 'y': np.nan}}
    pose.append(unknow_obj)
    return pose

In [125]:
def generate_data_posenet(posenetPred):
    '''
    This functions generates l2 normalized data form non-empty(Must have atleast 1 humans)
    Posenet results
    
    Parameters
    ---------
    posenetPred : The non-empty output of Posenet model
    Example {'detectionList': [{keypoints:[{score: , part:.. ,position:..}]} , human2 , ....]
    
    Return
    ------
    normalized_data : A 2D list with exte
    '''
    data = []
    for human in posenetPred['detectionList']:
        extended_pose = get_posenet_extended_pose(human['keypoints'])
        data.append(extended_pose)

    normalized_data = generate_dataset(data=data)
    return normalized_data
    

In [126]:
#generate_data_posenet(p)

array([[0.18694303, 0.11572664, 0.18694303, 0.11572664, 0.19830082,
        0.12155902, 0.20566803, 0.13230287, 0.19983565, 0.13260983,
        0.18264549, 0.12217295, 0.19615205, 0.10774549, 0.19185451,
        0.10958729, 0.20198442, 0.14335369, 0.21242131, 0.16054385,
        0.21088647, 0.17834795, 0.19031967, 0.14396762, 0.18755697,
        0.16115779, 0.18510123, 0.17834795, 0.18663606, 0.11388483,
        0.18540819, 0.11480574, 0.1909336 , 0.11449877, 0.18356639,
        0.11572664],
       [0.22652992, 0.04472884, 0.22652992, 0.04472884, 0.23198075,
        0.04761457, 0.2332633 , 0.0522638 , 0.2332633 , 0.05899718,
        0.22572833, 0.04841616, 0.22652992, 0.05242412, 0.22604897,
        0.05579081, 0.23550775, 0.05547017, 0.23214107, 0.06236386,
        0.23230139, 0.07005915, 0.23133948, 0.05579081, 0.23021725,
        0.06252418, 0.23166011, 0.06941787, 0.22685056, 0.04408756,
        0.2263696 , 0.04408756, 0.22797279, 0.04472884, 0.22556801,
        0.04488916],
      

In [13]:
#a = read_dataset(norm=True)

In [14]:
#a.columns.tolist()

['0_x',
 '0_y',
 '1_x',
 '1_y',
 '2_x',
 '2_y',
 '3_x',
 '3_y',
 '4_x',
 '4_y',
 '5_x',
 '5_y',
 '6_x',
 '6_y',
 '7_x',
 '7_y',
 '8_x',
 '8_y',
 '9_x',
 '9_y',
 '10_x',
 '10_y',
 '11_x',
 '11_y',
 '12_x',
 '12_y',
 '13_x',
 '13_y',
 '14_x',
 '14_y',
 '15_x',
 '15_y',
 '16_x',
 '16_y',
 '17_x',
 '17_y',
 'label']

In [34]:
#a.to_csv('/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPoses.csv', index=False)
#df_normalized_df = pd.DataFrame(df_normalized, columns=df.columns)

In [22]:
#df_normalized_df

In [39]:
#df_normalized_df.to_csv('/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPosesL2Normalized.csv', index=False)